In [87]:
import pandas as pd
import requests
import json
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [88]:
# Load training and validation datasets
zone_1_training_data = pd.read_csv("seismic_zone_1.csv")
zone_1_val_data = pd.read_csv("seismic_zone_1_val.csv")

zone_2_training_data = pd.read_csv("seismic_zone_2.csv")
zone_2_val_data = pd.read_csv("seismic_zone_2_val.csv")

In [ ]:
# Define function for loading validation data
FEATURES = ["depth", "mag", "gap", "dmin", "rms"]
def load_val_data(df):
    X_test = df[FEATURES] 
    y_true = df[["latitude", "longitude"]]
    
    return X_test, y_true

# API calls
def predict(url, X_test):
    API_URL = f'{url}/predict'

    predictions = []

    for _, row in X_test.iterrows():
        payload = row.to_dict()
        
        response = requests.post(API_URL, json=payload)
        
        if response.status_code == 200:
            pred = response.json()["prediction"]
            predictions.append(pred[0])
        else:
            print(f"Error for input {row}: {response.text}")
            
    return predictions

def evaluate_predictions(model_response, true_value):
    y_pred = pd.DataFrame(model_response, columns=["latitude", "longitude"])
    
    y_pred = pd.DataFrame(model_response, columns=["latitude", "longitude"])
    true_value = true_value.reset_index(drop=True)

    mse_lat = mean_squared_error(true_value["latitude"], y_pred["latitude"])
    mse_lon = mean_squared_error(true_value["longitude"], y_pred["longitude"])

    r2_lat = r2_score(true_value["latitude"], y_pred["latitude"])
    r2_lon = r2_score(true_value["longitude"], y_pred["longitude"])

    mae_lat = mean_absolute_error(true_value["latitude"], y_pred["latitude"])
    mae_lon = mean_absolute_error(true_value["longitude"], y_pred["longitude"])

    print(f"Mean Squared Error (Latitude): {mse_lat}")
    print(f"Mean Squared Error (Longitude): {mse_lon}\n")

    print(f"R² Score (Latitude): {r2_lat}")
    print(f"R² Score (Longitude): {r2_lon}\n")

    print(f"Mean Absolute Error (Latitude): {mae_lat}")
    print(f"Mean Absolute Error (Longitude): {mae_lon}")
    
def retrain(url, df, save_as="updated_model.pkl"):
    API_URL = f'{url}/retrain'
    
    data_payload = {
        "features": df[["depth", "mag", "gap", "dmin", "rms"]].to_dict(orient="records"),
        "latitude": df["latitude"].tolist(),
        "longitude": df["longitude"].tolist()
    }

    params = {"save_as": save_as}
    response = requests.post(API_URL, params=params, json=data_payload)
    print(response.json())
    
def list_models(url):
    print(requests.get(f'{url}/list_models').json())
    
def reload_model(url, model_name):
    API_URL = f'{url}/reload_model'
    params = {"model_name": model_name}

    response = requests.post(API_URL, params=params)
    print(response.json())
    
def download_model(url, model_name):
    API_URL = f'{url}/download_model'
    params = {"model_name": model_name}
    
    response = requests.get(API_URL, params=params)
    
    if response.status_code == 200:
        with open(model_name, "wb") as file:
            file.write(response.content)
        print(f"✅ {model_name} downloaded successfully!")
    else:
        print(f"❌ Failed to download {model_name}. Error: {response.json()}")
    
def upload_model(url, model_file):
    API_URL = f'{url}/upload_model'

    with open(model_file, "rb") as file:
        response = requests.post(API_URL, files={"file": file})
        
    print(response.json())

# Zone 1

In [90]:
zone1_API = "https://fedml.onrender.com"
X_1, y_1 = load_val_data(zone_1_val_data)

In [91]:
list_models(zone1_API)

{'saved_models': ['zone_0_zone_1_model.pkl', 'zone_0_model.pkl']}


In [92]:
predictions = predict(zone1_API, X_1)

In [93]:
evaluate_predictions(predictions, y_1)

Mean Squared Error (Latitude): 55.6867846556177
Mean Squared Error (Longitude): 22.85913167833489

R² Score (Latitude): -5.6006510125206415
R² Score (Longitude): -21.468817233732548

Mean Absolute Error (Latitude): 6.683537563025212
Mean Absolute Error (Longitude): 4.655139546218472


In [94]:
retrain(zone1_API, zone_1_training_data, 'zone_0_zone_1_model.pkl')

{'message': 'Model retrained and saved as zone_0_zone_1_model.pkl.'}


In [95]:
list_models(zone1_API)

{'saved_models': ['zone_0_zone_1_model.pkl', 'zone_0_model.pkl']}


In [96]:
reload_model(zone1_API, 'zone_0_zone_1_model.pkl')

{'message': '✅ Model zone_0_zone_1_model.pkl successfully reloaded!'}


In [97]:
predictions_1 = predict(zone1_API, X_1)

In [98]:
evaluate_predictions(predictions, y_1)

Mean Squared Error (Latitude): 55.6867846556177
Mean Squared Error (Longitude): 22.85913167833489

R² Score (Latitude): -5.6006510125206415
R² Score (Longitude): -21.468817233732548

Mean Absolute Error (Latitude): 6.683537563025212
Mean Absolute Error (Longitude): 4.655139546218472


In [99]:
download_model(zone1_API, 'zone_0_zone_1_model.pkl')

✅ zone_0_zone_1_model.pkl downloaded successfully!


# Zone 2

In [100]:
zone2_API = "https://fedml-zone2.onrender.com"
X_2, y_2 = load_val_data(zone_2_val_data)

In [101]:
list_models(zone2_API)

{'saved_models': ['zone_0_model.pkl', 'zone_0_zone_1_model.pkl']}


In [102]:
predictions_2 = predict(zone2_API, X_2)

In [103]:
evaluate_predictions(predictions_2, y_2)

Mean Squared Error (Latitude): 8.581386455635915
Mean Squared Error (Longitude): 1.419967758709142

R² Score (Latitude): -12.975120181897791
R² Score (Longitude): -0.03546827145377596

Mean Absolute Error (Latitude): 2.598107553333334
Mean Absolute Error (Longitude): 0.8375822466666637


In [104]:
retrain(zone2_API, zone_2_training_data, "zone_0_zone_2_model.pkl")

{'message': 'Model retrained and saved as zone_0_zone_2_model.pkl.'}


In [105]:
list_models(zone2_API)

{'saved_models': ['zone_0_model.pkl', 'zone_0_zone_1_model.pkl', 'zone_0_zone_2_model.pkl']}


In [106]:
reload_model(zone2_API, "zone_0_zone_2_model.pkl")

{'message': '✅ Model zone_0_zone_2_model.pkl successfully reloaded!'}


In [107]:
predictions_2 = predict(zone2_API, X_2)

In [108]:
evaluate_predictions(predictions_2, y_2)

Mean Squared Error (Latitude): 0.28959644008745367
Mean Squared Error (Longitude): 0.6505956206239898

R² Score (Latitude): 0.5283809818617452
R² Score (Longitude): 0.5255729444763328

Mean Absolute Error (Latitude): 0.36367409333333334
Mean Absolute Error (Longitude): 0.4909193933333293


In [109]:
download_model(zone2_API, 'zone_0_zone_2_model.pkl')

✅ zone_0_zone_2_model.pkl downloaded successfully!


# Aggregation and Deployment of New Model

In [110]:
from sklearn.ensemble import RandomForestRegressor
import joblib

In [112]:
zone_0_model = joblib.load("zone_0_model.pkl")
zone_1_model = joblib.load("zone_0_zone_1_model.pkl")
zone_2_model = joblib.load("zone_0_zone_2_model.pkl")

Federated Decision Boundary Approximation completed. Model saved as zone_0+1+2_model.pkl


In [142]:
upload_model(zone1_API, 'zone_0+1+2_model.pkl')
upload_model(zone2_API, 'zone_0+1+2_model.pkl')

{'message': '✅ Model zone_0+1+2_model.pkl uploaded successfully!'}
{'message': '✅ Model zone_0+1+2_model.pkl uploaded successfully!'}


In [143]:
list_models(zone1_API)
list_models(zone2_API)

{'saved_models': ['zone_0_zone_1_model.pkl', 'zone_0_model.pkl', 'zone_0+1+2_model.pkl']}
{'saved_models': ['zone_0_model.pkl', 'zone_0_zone_1_model.pkl', 'zone_0+1+2_model.pkl']}


# Testing on new aggregated model for Zone 1 and Zone 2

In [145]:
reload_model(zone1_API, 'zone_0+1+2_model.pkl')
reload_model(zone2_API, 'zone_0+1+2_model.pkl')

{'error': 'Model zone_0+1+2_model.pkl not found!'}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [133]:
#Zone 1
predictions_1_agg = predict(zone1_API, X_1)

In [135]:
print(predictions_1_agg)

[111.0, 94.0, 120.0, 91.0, 123.0, 114.0, 155.0, 118.0, 97.0, 142.0, 80.0, 111.0, 76.0, 160.13063934326172, 136.0, 116.0, 81.0, 128.0, 112.0, 139.0, 116.0, 103.0, 89.0, 149.0, 116.0, 80.0, 99.0, 140.0, 97.0, 146.0, 118.0, 84.0, 111.0, 123.0, 126.0, 120.0, 115.0, 129.0, 116.0, 93.0, 115.0, 114.0, 112.0, 119.0, 155.0, 126.0, 71.0, 112.0, 106.0, 121.0, 100.0, 110.0, 76.0, 101.0, 130.0, 145.0, 122.0, 85.0, 122.0, 113.0, 116.0, 115.0, 151.0, 111.0, 122.0, 123.0, 145.0, 136.0, 132.0, 35.0, 95.0, 99.0, 97.0, 110.0, 115.0, 95.0, 97.0, 94.0, 105.0, 42.81885059356689, 118.0, 119.0, 124.0, 126.0, 31.53499984741211, 119.0, 46.254440269470216, 108.0, 133.0, 121.0, 125.0, 72.0, 136.0, 103.0, 65.0, 28.0, 97.0, 133.0, 132.0, 117.0, 118.0, 116.0, 106.0, 105.0, 88.0, 130.0, 70.0, 157.0, 105.0, 152.0, 120.0, 134.0, 138.0, 88.0, 114.0, 84.0, 111.0, 104.0, 127.0]


In [ ]:
evaluate_predictions(predictions_1_agg, y_1)

In [ ]:
# Zone 2
predictions_2_agg = predict(zone2_API, X_2)

In [137]:
print(predictions_2_agg)

[121.0, 118.0, 59.0, 26.22, 117.0, 121.0, 115.0, 125.0, 114.0, 140.0, 123.0, 119.0, 154.0, 44.0, 128.0, 85.0, 73.0, 120.4857763671875, 122.0, 43.0, 76.0, 114.0, 125.0, 62.0, 120.0, 81.0, 137.0, 118.0, 111.0, 590.2712957763672, 85.0, 60.58656005859375, 89.0, 100.0, 116.0, 112.0, 110.0, 136.0, 117.0, 118.0, 122.0, 73.0, 74.0, 106.0, 67.0, 118.0, 136.0, 92.89970146179199, 114.0, 115.0, 140.0, 89.0, 107.0, 106.0, 103.0, 118.0, 127.0, 60.0, 132.0, 121.0, 113.0, 93.0, 79.85814071655274, 115.0, 70.0, 61.0, 200.94717239379884, 108.0, 110.0, 90.0, 67.0, 119.0, 445.4386068725586, 140.0, 140.0, 115.0, 66.0, 128.0, 82.0, 85.0, 115.0, 120.0, 135.0, 93.0, 134.0, 563.75, 79.0, 112.0, 107.0, 80.0, 95.0, 66.0, 118.0, 125.0, 155.0, 118.0, 126.0, 83.0, 160.0, 114.0, 101.0, 133.0, 87.0, 104.0, 58.0, 174.0, 110.0, 141.0, 49.84299850463867, 86.0, 126.0, 125.0, 105.0, 122.0, 85.0, 118.0, 72.0, 86.0, 82.8952904510498, 77.0, 118.0, 588.0885009765625, 41.0, 106.0, 114.0, 124.0, 541.3823291015625, 72.0, 135.0, 6

In [ ]:
evaluate_predictions(predictions_2_agg, y_2)